# Proyecto 14

El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- la calidad de la predicción;
- la velocidad de la predicción;
- el tiempo requerido para el entrenamiento

## Preparación de datos

In [194]:
#Importar librerias
import math 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression 
from sklearn.ensemble import RandomForestRegressor

In [4]:
#leer el dataset
df= pd.read_csv('car_data.csv')

In [5]:
df.head(5)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,24/03/2016 11:52,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,24/03/2016 00:00,0,70435,07/04/2016 03:16
1,24/03/2016 10:58,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,24/03/2016 00:00,0,66954,07/04/2016 01:46
2,14/03/2016 12:52,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,14/03/2016 00:00,0,90480,05/04/2016 12:47
3,17/03/2016 16:54,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,17/03/2016 00:00,0,91074,17/03/2016 17:40
4,31/03/2016 17:25,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,31/03/2016 00:00,0,60437,06/04/2016 10:17


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Mileage            354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

In [126]:
#Preprocesamiento de datos, limpiar las columnas que no son necesarias para el entrenamiento del modelo 
df_clean= df.drop(['DateCrawled','DateCreated','PostalCode','LastSeen','NumberOfPictures'],axis=1)

In [8]:
df_clean['RegistrationYear'].value_counts().sort_index()

RegistrationYear
1000    37
1001     1
1039     1
1111     3
1200     1
        ..
9000     3
9229     1
9450     1
9996     1
9999    26
Name: count, Length: 151, dtype: int64

In [128]:
# Selecionar los registros anuales dentro del lapso de fechas
df_clean= df_clean.query('RegistrationYear > 1900 and RegistrationYear < 2025').reset_index(drop=True)

In [10]:
df_clean

,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired
0,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN
1,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no
...,...,...,...,...,...,...,...,...,...,...,...
354193,0,NaN,2005,manual,0,colt,150000,7,petrol,mitsubishi,yes
354194,2200,NaN,2005,NaN,0,NaN,20000,1,NaN,sonstige_autos,NaN
354195,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no
354196,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no


In [11]:
df_clean.describe()

,Price,RegistrationYear,Power,Mileage,RegistrationMonth
count,354198.000000,354198.000000,354198.000000,354198.000000,354198.000000
mean,4417.651314,2003.084789,110.078242,128267.607383,5.716819
std,4514.081022,7.536418,189.536766,37823.538557,3.725539
min,0.000000,1910.000000,0.000000,5000.000000,0.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000
max,20000.000000,2019.000000,20000.000000,150000.000000,12.000000


In [130]:
# La columna power deben ser valores ssuperiores al 0 
df_clean =df_clean.query('Power > 0').reset_index(drop=True)

In [13]:
df_clean.duplicated().sum()

24353

In [132]:
#Eliminar los duplicados
df_clean.drop_duplicates(inplace=True)

In [15]:
df_clean.isna().sum()

Price                    0
VehicleType          21890
RegistrationYear         0
Gearbox               6271
Power                    0
Model                12899
Mileage                  0
RegistrationMonth        0
FuelType             20601
Brand                    0
NotRepaired          47103
dtype: int64

In [134]:
#Rellenar los valores nulos
df_clean.fillna('desconocido', inplace=True)

In [136]:
df_clean.isna().sum()

Price                0
VehicleType          0
RegistrationYear     0
Gearbox              0
Power                0
Model                0
Mileage              0
RegistrationMonth    0
FuelType             0
Brand                0
NotRepaired          0
dtype: int64

In [138]:
#Definir las variables categoricas 
variables_categoricas=['VehicleType','Gearbox','Model','FuelType','Brand','NotRepaired']

In [148]:
#Crer un bucle para iterar dentro de los valores de cada variable categorica
for variable_categorica in variables_categoricas:
    print(df_clean[variable_categorica].value_counts())
    print()

VehicleType
sedan          77709
small          65660
wagon          55422
bus            24663
desconocido    21890
convertible    17627
coupe          14080
suv            10150
other           2546
Name: count, dtype: int64

Gearbox
manual         226965
auto            56511
desconocido      6271
Name: count, dtype: int64

Model
golf                  24299
other                 20146
3er                   16664
desconocido           12899
polo                  10462
                      ...  
samara                    5
kalina                    4
rangerover                3
serie_3                   3
range_rover_evoque        1
Name: count, Length: 250, dtype: int64

FuelType
petrol         181051
gasoline        82699
desconocido     20601
lpg              4547
cng               471
hybrid            202
other              99
electric           77
Name: count, dtype: int64

Brand
volkswagen        62757
opel              32010
bmw               31006
mercedes_benz     26457
aud

In [142]:
# Convertir las varables categoricas en variales numericas
df_clean_one_hot=pd.get_dummies(df_clean, drop_first=True)

In [144]:
df_clean_one_hot

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,VehicleType_convertible,VehicleType_coupe,VehicleType_desconocido,VehicleType_other,VehicleType_sedan,...,Brand_smart,Brand_sonstige_autos,Brand_subaru,Brand_suzuki,Brand_toyota,Brand_trabant,Brand_volkswagen,Brand_volvo,NotRepaired_no,NotRepaired_yes
0,18300,2011,190,125000,5,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,True
1,9800,2004,163,125000,8,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,1500,2001,75,150000,6,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False
3,3600,2008,69,90000,7,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
4,650,1995,102,150000,10,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314095,5250,2016,150,150000,12,False,False,True,False,False,...,False,False,False,False,False,False,False,False,True,False
314096,3200,2004,225,150000,5,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,True
314097,1199,2000,101,125000,3,True,False,False,False,False,...,True,False,False,False,False,False,False,False,True,False
314098,9200,1996,102,150000,3,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,False


## Entrenamiento del modelo 

In [150]:
#Dividir el conjunto de entrenamieto, validacion y prueba
train_valid_oh, test_oh = train_test_split(df_clean_one_hot, test_size=0.2, random_state=12345)

In [151]:
train_oh, valid_oh = train_test_split(train_valid_oh, test_size=0.2, random_state=12345)

In [154]:
#Definir las caracteriscas y el objetivo para el conjunto de entrenamiento
train_oh_caracteristicas = train_oh.drop('Price', axis=1)
train_oh_objetivo = train_oh['Price']

In [179]:
#Definir las caracteriscas y el objetivo para el conjunto de validacion
valid_oh_caracteristicas = valid_oh.drop('Price', axis=1)
valid_oh_objetivo = valid_oh['Price']

In [156]:
#Definir las caracteriscas y el objetivo para el conjunto de prueba
test_oh_caracteristicas = test_oh.drop('Price', axis=1)
test_oh_objetivo = test_oh['Price']

In [158]:
#Definir una funcion que calcule del rmse el valor real y la prediccion
def rmse(real, prediccion):
    return math.sqrt(mean_squared_error(real, prediccion))

In [160]:
#Entrenar el modelo del Regresion Lineal 
modelo_rl = LinearRegression()

In [162]:
modelo_rl.fit(train_oh_caracteristicas, train_oh_objetivo)

LinearRegression()

In [164]:
#Calcular las predicciones por cada conjunto de datos 
pred_train= modelo_rl.predict(train_oh_caracteristicas)
pred_valid= modelo_rl.predict(valid_oh_caracteristicas)
pred_test= modelo_rl.predict(test_oh_caracteristicas)

In [166]:
#Imprimir los resultados 
print('RMSE entrenamiento: ', rmse(train_oh_objetivo, pred_train))
print('RMSE validacion: ', rmse(valid_oh_objetivo, pred_valid))
print('RMSE prueba: ', rmse(test_oh_objetivo, pred_test))

RMSE entrenamiento:  2836.9224930052465
RMSE validacion:  2827.646114363709
RMSE prueba:  2865.203943866324


### Observaciones
Los valores de RMSE de cada conjunto son muy similares, lo que significa que el modelo tiene un buen rendimiento y generaliza bien sin caer en sobreajuste.

In [169]:
#Probar modelo con Bosque de Rgresion
modelo_rf = RandomForestRegressor(n_estimators= 100, max_depth= None, verbose = True, n_jobs=4)

In [171]:
#Entrenar el modelo 
modelo_rf.fit(train_oh_caracteristicas, train_oh_objetivo)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:  2.6min
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:  5.9min finished


RandomForestRegressor(n_jobs=4, verbose=True)

In [77]:
pred_train_rf= modelo_rf.predict(train_oh_caracteristicas)
pred_valid_rf= modelo_rf.predict(valid_oh_caracteristicas)
pred_test_rf= modelo_rf.predict(test_oh_caracteristicas)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    6.5s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   16.6s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    3.6s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    9.7s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    4.6s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   11.2s finished


In [78]:
print('RMSE entrenamiento: ', rmse(train_oh_objetivo, pred_train_rf))
print('RMSE validacion: ', rmse(valid_oh_objetivo, pred_valid_rf))
print('RMSE prueba: ', rmse(test_oh_objetivo, pred_test_rf))

RMSE entrenamiento:  780.9240192774491
RMSE validacion:  1763.2533195340934
RMSE prueba:  1787.8556790517453


#### Observaciones
El modelo de Bosque de Regresion parece sobreajustado porque funciona muy bien en datos de entrenamiento pero no en el conjunto de validacion. Ademas que para entrenar el modelo y hacer las predicciones toma mucho tiempo mas de lo esperado y no es el mejor modelo en cuento a rapidez versus calidad. 

## Análisis del modelo

In [88]:
!pip install catboost

   ---------------------------------------- 0.0/102.4 MB ? eta -:--:--
   ---------------------------------------- 0.1/102.4 MB 1.6 MB/s eta 0:01:04
   ---------------------------------------- 0.5/102.4 MB 5.6 MB/s eta 0:00:19
    --------------------------------------- 1.3/102.4 MB 10.6 MB/s eta 0:00:10
   - -------------------------------------- 2.6/102.4 MB 15.2 MB/s eta 0:00:07
   - -------------------------------------- 4.0/102.4 MB 18.2 MB/s eta 0:00:06
   -- ------------------------------------- 5.3/102.4 MB 20.1 MB/s eta 0:00:05
   -- ------------------------------------- 6.8/102.4 MB 21.8 MB/s eta 0:00:05
   --- ------------------------------------ 8.3/102.4 MB 23.0 MB/s eta 0:00:05
   --- ------------------------------------ 9.6/102.4 MB 23.6 MB/s eta 0:00:04
   ---- ----------------------------------- 11.0/102.4 MB 29.7 MB/s eta 0:00:04
   ---- ----------------------------------- 12.5/102.4 MB 29.8 MB/s eta 0:00:04
   ----- ---------------------------------- 13.8/102.4 MB 29

In [175]:
# Importar el modelo CatBoostRegresor para mejor el modelo por medio de la potenciacion de gradiente
from catboost import CatBoostRegressor

In [122]:
model_catboost = CatBoostRegressor(
    iterations = 1000,
    learning_rate = 0.2
)

In [177]:
#Entrenar el modelo 
model_catboost.fit(train_oh_caracteristicas, train_oh_objetivo, eval_set=(valid_oh_caracteristicas,valid_oh_objetivo))

0:	learn: 4036.3821721	test: 4031.4650027	best: 4031.4650027 (0)	total: 152ms	remaining: 2m 32s
1:	learn: 3591.9022986	test: 3585.6902824	best: 3585.6902824 (1)	total: 190ms	remaining: 1m 34s
2:	learn: 3264.5430231	test: 3258.1228444	best: 3258.1228444 (2)	total: 215ms	remaining: 1m 11s
3:	learn: 3006.1842500	test: 3000.0376744	best: 3000.0376744 (3)	total: 238ms	remaining: 59.4s
4:	learn: 2817.3395569	test: 2809.6759077	best: 2809.6759077 (4)	total: 260ms	remaining: 51.8s
5:	learn: 2672.1916386	test: 2662.6300603	best: 2662.6300603 (5)	total: 283ms	remaining: 46.8s
6:	learn: 2548.4327098	test: 2539.9164855	best: 2539.9164855 (6)	total: 305ms	remaining: 43.3s
7:	learn: 2458.7155639	test: 2450.8693106	best: 2450.8693106 (7)	total: 327ms	remaining: 40.6s
8:	learn: 2389.4264470	test: 2382.7642153	best: 2382.7642153 (8)	total: 349ms	remaining: 38.4s
9:	learn: 2338.0095922	test: 2331.0014398	best: 2331.0014398 (9)	total: 378ms	remaining: 37.4s
10:	learn: 2286.9979396	test: 2279.7087555	best

In [181]:
#Realizar las predicciones
pred_train_ct= model_catboost.predict(train_oh_caracteristicas)
pred_valid_ct= model_catboost.predict(valid_oh_caracteristicas)
pred_test_ct=  model_catboost.predict(test_oh_caracteristicas)

In [183]:
#Imprimir los resultados
print('RMSE entrenamiento: ', rmse(train_oh_objetivo, pred_train_ct))
print('RMSE validacion: ', rmse(valid_oh_objetivo, pred_valid_ct))
print('RMSE prueba: ', rmse(test_oh_objetivo, pred_test_ct))

RMSE entrenamiento:  1549.4052283955627
RMSE validacion:  1668.9409945393515
RMSE prueba:  1699.9339859818429


In [185]:
#Probar un nuevo modelo catboostregresor pero sin transformarcion one hot
entrenamiento_validacion, prueba = train_test_split(df_clean, test_size=0.20)

In [187]:
#Dividir los conjuntos de datos 
entrenamiento, validacion = train_test_split(entrenamiento_validacion, test_size=0.20)

In [189]:
#Seleccionar las caracteristicas y los objetivos por cada conjunto de datos
entrenamiento_features = entrenamiento.drop(['Price'], axis=1) 
valid_features = validacion.drop(['Price'], axis=1)
test_features = prueba.drop(['Price'], axis=1)

In [110]:
entrenamiento_obj = entrenamiento['Price'] 
valid_obj = validacion['Price']
test_obj = prueba['Price']

In [114]:
model_catboost_2 = CatBoostRegressor(
    iterations = 1000,
    learning_rate = 0.1,
    cat_features = variables_categoricas 
)

In [116]:
#Entrenar el modelo 
model_catboost_2.fit(entrenamiento_features, entrenamiento_obj, eval_set=(valid_features, valid_obj))

0:	learn: 4299.2297666	test: 4320.0423270	best: 4320.0423270 (0)	total: 295ms	remaining: 4m 54s
1:	learn: 4037.7691193	test: 4058.9695971	best: 4058.9695971 (1)	total: 558ms	remaining: 4m 38s
2:	learn: 3820.1946066	test: 3840.9501755	best: 3840.9501755 (2)	total: 987ms	remaining: 5m 28s
3:	learn: 3625.2504458	test: 3645.9211567	best: 3645.9211567 (3)	total: 1.24s	remaining: 5m 7s
4:	learn: 3444.9060511	test: 3466.2383028	best: 3466.2383028 (4)	total: 1.46s	remaining: 4m 50s
5:	learn: 3291.3993549	test: 3311.9642088	best: 3311.9642088 (5)	total: 1.69s	remaining: 4m 39s
6:	learn: 3149.1496583	test: 3170.7278779	best: 3170.7278779 (6)	total: 1.9s	remaining: 4m 30s
7:	learn: 3030.2158930	test: 3052.7546186	best: 3052.7546186 (7)	total: 2.11s	remaining: 4m 21s
8:	learn: 2912.9083795	test: 2935.8865051	best: 2935.8865051 (8)	total: 2.34s	remaining: 4m 17s
9:	learn: 2819.4573075	test: 2842.2726155	best: 2842.2726155 (9)	total: 2.54s	remaining: 4m 11s
10:	learn: 2736.5924793	test: 2759.5416188

### Conclusiones
El Mejoramiento del modelo para obtener la mejor prediccion y que no sacrifique gasto computacional y tiempo apesar de sobreajustar los hiperametros es el modelo de potenciacion de gradiente CatBoostRegresor codificando las variables categorias con one-hot, se podria seguir optimizando el model con mas iteraciones pero el tiempo y el gasto computaciones serian mayor pero con una precision aun mayor. 

# Lista de control

Escribe 'x' para verificar. Luego presiona Shift+Enter

- [x]  Jupyter Notebook está abierto
- [ ]  El código no tiene errores- [ ]  Las celdas con el código han sido colocadas en orden de ejecución- [ ]  Los datos han sido descargados y preparados- [ ]  Los modelos han sido entrenados
- [ ]  Se realizó el análisis de velocidad y calidad de los modelos